In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from sample.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from sample.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter
from IPython.display import clear_output
from sample.notebooks.analysis.data_info import *


In [2]:
exp = get_exp((40,262,287),directory_project)


begin = 2020-12-26 20:04:00 
  end = 2020-12-30 23:33:00


In [3]:
exp.load_compressed_skel()

In [105]:
directory = directory_project

plate_number = 94
plate = get_postion_number(plate_number)
begin = 269
end = 329
dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime[begin:end+1]
exp2 = Experiment(plate,directory)
exp2.load(dates) #for method 2

In [3]:
def get_time(dates_datetimes,i,j):
    seconds = (dates_datetimes[j]-dates_datetimes[i]).total_seconds()
    return(seconds/3600)

In [3]:
inst = (39,269,329)
directory = directory_project
plate,begin,end = inst
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_pruned_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

In [13]:
i=10
boundaries=(2800,3700,3100,4200)

to_plot = ims[i][boundaries[0]:boundaries[1],boundaries[2]:boundaries[3]]
plot_t_tp1([], [], None, None, to_plot*0, to_plot,time=f't = {int(get_time(dates_datetime_chosen,0,i))}h')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
start=0
finish = end-begin
boundaries=(2800,3700,3100,4200)
for i in range(start,finish):
    to_plot = ims[i][boundaries[0]:boundaries[1],boundaries[2]:boundaries[3]]
    plt.close('all')
    clear_output(wait=True)
    plot_t_tp1([], [], None, None, to_plot*0, to_plot, save=f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/im{i}',time=f't = {int(get_time(dates_datetime_chosen,0,i))}h')
img_array = []
for t in range(start,finish):
    img = cv2.imread(f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/im{t}.png')
    img_array.append(img)
plate_label = plate_number[inst] 
imageio.mimsave(f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/movie_close_up{plate_label}temp_ {begin}_{end}_{dates[start]}_{dates[finish]}.gif', img_array,duration = 1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plate_label = plate_number[inst] 
imageio.mimsave(f'/home/cbisot/pycode/MscThesis/sample/notebooks/plotting/Figure/movie_close_up{plate_label}temp_ {begin}_{end}_{dates[start]}_{dates[finish]}.gif', img_array,duration = 1)

In [3]:
RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)

In [4]:
problems = []
for rh in RH:
    roots = []
    hyph = rh
    tip  =hyph.end
    for t in tip.ts():
        #             print(t,tip)
        if tip.degree(t) == 1:
            root, edges, nodes = hyph.get_edges(t, 200)
            roots.append(root)
    occurence_count = Counter(roots)
    if occurence_count.most_common(2)[0][0]!=rh.root:
        problems.append(rh)

In [5]:
len(problems),len(RH)

(17, 175)

In [6]:
rh = choice(problems)
rh

Hyphae(8135,8193)

In [9]:
rh.ts,rh.end.ts(),occurence_count.most_common(2)[0][0].ts()

([21, 22], [21, 22, 23, 24, 25], [22, 23, 24, 25])

In [8]:
roots = []
hyph = rh
tip  =hyph.end
for t in rh.end.ts():
    #             print(t,tip)
    if tip.degree(t) == 1:
        root, edges, nodes = hyph.get_edges(t, 200)
        roots.append(root)
occurence_count = Counter(roots)
occurence_count

Counter({Node(8138): 1, Node(9062): 1, Node(8999): 2})

In [11]:
plt.close('all')
plot_raw_plus(exp,rh.ts[0]+1,[rh.end.label,rh.root.label]+[occurence_count.most_common(2)[0][0].label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
exp.plot([rh.ts[0]],[[rh.end.label,rh.root.label]+[occurence_count.most_common(2)[0][0].label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
rh = choice(select)
rh.mother,rh,rh.ts

NameError: name 'select' is not defined

In [7]:
roots = []
hyph = rh.mother[0]
tip  =Node(hyph.end.label,exp2)
hypha = Hyphae(tip)
for t in tip.ts():
    #             print(t,tip)
    if tip.degree(t) == 1:
        root, edges, nodes = hypha.get_edges(t, 200)
        roots.append(root)
occurence_count = Counter(roots)
occurence_count

NameError: name 'exp2' is not defined

In [8]:
roots = []
hyph = rh.mother[0]
tip  =hyph.end
for t in tip.ts():
    #             print(t,tip)
    if tip.degree(t) == 1:
        root, edges, nodes = hyph.get_edges(t, 200)
        roots.append(root)
occurence_count = Counter(roots)
occurence_count

Counter({Node(5243): 2, Node(4144): 1})

In [6]:
len(problems)

48

In [5]:
len(problems)

0

In [3]:
from sample.pipeline.functions.hyphae_id_surf import (
    clean_and_relabel,
    get_mother,
    save_hyphaes,
    resolve_ambiguity_two_ends,
    clean_obvious_fake_tips,
    width_based_cleaning
)
exp_clean = clean_and_relabel(exp)
RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp_clean,criter)

0.0
0.055233360950013806
0.11046672190002761
0.16570008285004142
0.22093344380005522
0.27616680475006905
0.33140016570008285
0.38663352665009665
0.44186688760011045
0.4971002485501243
0.5523336095001381
0.6075669704501518
0.6628003314001657
0.7180336923501796
0.7732670533001933
0.8285004142502072
0.8837337752002209
0.9389671361502347
0.9942004971002486
Detected problems during hyphae detection, 83 hyphaes have inconsistent root over time
removing  210  nodes out of 8985 because they appear only in one timestep and are not within an identified hypha
[45.888564]
[1.136003, 37.595403]
[57.012361]
[61.589241]
[69.797028]
[72.570175]
[79.39028]
[82.292056]
[88.018915]
[93.215148]
[94.431121]
[102.698136]
[107.651592]
[114.85416]
[124.033512]
[135.147506]
[142.897725]
[149.376738]
[152.434632]
[161.84289]
[173.575468]
[188.503416]
[191.833005]
[206.577216]
[220.747472]
[239.920172]
[264.566085]
[273.541532]
[297.374064]
[318.481112]
[339.248447]
[361.865686]
[383.728288]
[407.257764]
[444.27

In [104]:
occurence_count

Counter({Node(4338): 26, Node(12705): 1})

In [99]:
hyph.root.ts()

[47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]

In [109]:
occurence_count.most_common(2)[0][0]

Node(4338)

In [112]:
Node(9144,exp).degree(50)

3

In [9]:
roots_new_branch = []
total_growth = 0
counts =[]
total_growths = []
lapses = []
for lapse in [8,7,6,5,4,3,2,1]:
    for rh in RH:
        for index in range(len(rh.ts)):
                if index+lapse<len(rh.ts):
                    t = rh.ts[index]
                    tp1 = rh.ts[index + lapse]
                    pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
                    count=len(nodes[1:-1]) 
                    total_growth = np.sum([get_length_um(seg) for seg in pixels])
                    counts.append(count)
                    total_growths.append(total_growth)
                    lapses.append(lapse)


In [127]:
len(lapses)

33465

In [28]:
plt.close('all')
plate = 94
data = {'y':  counts,
        'x': lapses,
       'z' : total_growths}
df = pd.DataFrame (data)
df['frequ']=df['y']/df['z']
abcisse = 'z'
ordinate = 'frequ'
tab = df
baits_sort = tab.sort_values(abcisse)
N=1500
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('alpha (°)')
ax.set_ylabel('angle between speed vector and density gradient(°)')
# ax.set_xlim(-40,40)
# ax.set_ylim(-60,60)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
ax.scatter(tab[abcisse],tab[ordinate],label=plate,alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
values = []
for lapse in [1,2,3,4,5,6,7,8]:
    aft  = df.loc[df['x']==lapse]
    value = np.mean(aft['y']/aft['z'])
    values.append(value)


In [30]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.plot([1,2,3,4,5,6,7,8],values)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …